In [1]:
%load_ext autoreload

%autoreload 2

In [3]:
import os
import json
from collections import defaultdict
import re

# !pip install smart_open
# !pip install orbax-checkpoint==0.2.6

os.environ["JAX_PLATFORMS"] = "cpu"
import smart_open
import numpy as np
import jax
import orbax
import orbax.checkpoint
from etils import epath
from flax.traverse_util import flatten_dict, unflatten_dict
from jax import ShapeDtypeStruct

# orbax-checkpoint=0.2.6需要修改下下面的代码: 将jax.config.jax_coordination_service去掉
# ~/miniconda3/lib/python3.10/site-packages/orbax/checkpoint/checkpoint_manager.py:334, in CheckpointManager.reached_preemption(self, step)
#     331 def reached_preemption(self, step: int) -> bool:
#     332   """Returns True if a preemption sync point has been reached."""
#     333   return (
# --> 334       jax.config.jax_coordination_service
#     335       and multihost_utils.reached_preemption_sync_point(step)
#     336   )

# AttributeError: 'Config' object has no attribute 'jax_coordination_service'


In [4]:
# no_prefix_2  : no_prefix元祖的第2个位置 
# m, v   <=> mdl_vars 
# p#12#i-1_2  : p#12#i-1元祖的第2个位置
# m, v
# opt 结构
# [{
# no_prefix: (count, count, count+mv, count)
# p#12#i-1: (count, count, count+mv, count)
# }]
# 最后一个count的shape 为 layer_num

# 构造mdl_vars和opt_states的shapedtype
layer_num = 48
sub_layer_num = 4
read_dir = "gs://llm_base_models_us-central2/v5p_256/7B/PileDCSlimLlama7B4Kx4x256x1v5p/checkpoints/"
# read_dir = "gs://llm_base_models/maxtext_align_pax_dc/PileDCSlimLlama7B32Kx4x256x1Align4/checkpoints/"
load_step = 440000

step_prefix = "checkpoint"
step_format_fixed_length = 8
options = orbax.checkpoint.CheckpointManagerOptions(step_prefix=step_prefix, step_format_fixed_length=step_format_fixed_length)
item = {"state": orbax.checkpoint.Checkpointer(orbax.checkpoint.PyTreeCheckpointHandler(use_ocdbt=False))}
pax_mngr = orbax.checkpoint.CheckpointManager(read_dir, item, options)

step_prefix = 'checkpoint'
checkpoint_name = f"{step_prefix}_" + str(load_step).zfill(step_format_fixed_length)
print(f"checkpoint_name: {checkpoint_name}")
metadata_path = os.path.join(read_dir, checkpoint_name, "metadata/metadata")
print(f"metadata_path: {metadata_path}")

with smart_open.open(metadata_path, "r") as f:
    metadata = json.load(f)

flat_metadata = flatten_dict(metadata["train_state_metadata"])
unpadded_global_shapes = defaultdict(dict)
for k, v in flat_metadata.items():
    param_key, shape_dtype = k[:-1], k[-1]
    if shape_dtype in ["unpadded_shape", "dtype"]:
        unpadded_global_shapes[param_key][shape_dtype] = v
    shape_dtype = unpadded_global_shapes[param_key]
    if len(shape_dtype) == 2:
        shape_dtype = jax.ShapeDtypeStruct(
            shape=shape_dtype["unpadded_shape"], dtype=shape_dtype["dtype"]
        )
        unpadded_global_shapes.update({param_key: shape_dtype})
unflat_unpadded_global_shapes = unflatten_dict(unpadded_global_shapes)

opt_unpadded_global_shapes = {}
first_count = {'count': ShapeDtypeStruct(shape = (), dtype = np.int32)}
last_count = {'count': ShapeDtypeStruct(shape = (layer_num // sub_layer_num, ), dtype = np.int32)}

transformer_shapedtype = {p: shapedtype for p, shapedtype in unpadded_global_shapes.items() if 'transformer' in p}
untransformer_shapedtype = {p: shapedtype for p, shapedtype in unpadded_global_shapes.items() if 'transformer' not in p}

transformer_shapedtype = unflatten_dict(transformer_shapedtype)
untransformer_shapedtype = unflatten_dict(untransformer_shapedtype)

untransformer_mvcount = {
    'count': ShapeDtypeStruct(shape = (), dtype = np.int32),
    'm': untransformer_shapedtype['mdl_vars'],
    'v': untransformer_shapedtype['mdl_vars']
          }
transformer_mvcount = {
    'count': ShapeDtypeStruct(shape = (), dtype = np.int32),
    'm': transformer_shapedtype['mdl_vars'],
    'v': transformer_shapedtype['mdl_vars']
          }

untransformer_values = (first_count, first_count, untransformer_mvcount, last_count)
transformer_values = (first_count, first_count, transformer_mvcount, last_count)

for i in range(4):
    opt_unpadded_global_shapes[f'no_prefix_{i}'] = untransformer_values[i]
    opt_unpadded_global_shapes[f'p#{layer_num // sub_layer_num}#i-1_{i}'] = transformer_values[i]
    
opt_unpadded_global_shapes = {'opt_states_0': opt_unpadded_global_shapes}

global_shapes = {
    "mdl_vars": unflat_unpadded_global_shapes["mdl_vars"], 
    "opt_states_0": opt_unpadded_global_shapes["opt_states_0"], 
    "step": ShapeDtypeStruct(shape = (), dtype = np.int32),
}

# 基于mdl_vars和opt_states的shapedtype加载模型
with jax.default_device(jax.devices("cpu")[0]):
    pax_state = pax_mngr.restore(load_step, items={'state':global_shapes})
flat_pax_state = {'.'.join(k): v for k, v in flatten_dict(pax_state).items()}

2024-07-05 03:15:27.159807: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


checkpoint_name: checkpoint_00440000
metadata_path: gs://llm_base_models_us-central2/v5p_256/7B/PileDCSlimLlama7B4Kx4x256x1v5p/checkpoints/checkpoint_00440000/metadata/metadata


ERROR:absl:Checkpoint structure file does not exist at gs://llm_base_models_us-central2/v5p_256/7B/PileDCSlimLlama7B4Kx4x256x1v5p/checkpoints/checkpoint_00440000/state. Attempting to assume an implicit tree structure.
I0000 00:00:1720149371.532901   21765 gcs_resource.cc:109] Using default AdmissionQueue with limit 32
I0000 00:00:1720149371.537039   22912 google_auth_provider.cc:180] Running on GCE, using service account 887571727717-compute@developer.gserviceaccount.com


In [5]:
# mgate 是小梦的模型，其他模型需要去掉
pax2max = f'''
pax_prefix_lm.embedding_lookup.emb_var (152064, 1024)
max_prefix_token_embedder.embedding (152064, 1024)

pax_prefix_lm.final_ln.scale (1024,)
max_prefix_decoder.decoder_norm.scale (1024,)

pax_prefix_lm.softmax.logits_ffn.linear.w (1024, 152064)
max_prefix_decoder.logits_dense.kernel (1024, 152064)

pax_prefix_lm.transformer.repeat.sub.x_layers_0.ff_layer.mgate_layer.linear.w (1024, 44)
max_prefix_decoder.layers.mlp_0.mgate.kernel (1024, 44)

pax_prefix_lm.transformer.repeat.sub.x_layers_0.ff_layer.ffn_layer1.linear.w (1024, 12, 2816)
max_prefix_decoder.layers.mlp_0.wi_1.kernel (1024, 12, 2816)

pax_prefix_lm.transformer.repeat.sub.x_layers_0.ff_layer.ffn_layer1_gate.linear.w (1024, 12, 2816)
max_prefix_decoder.layers.mlp_0.wi_0.kernel (1024, 12, 2816)

pax_prefix_lm.transformer.repeat.sub.x_layers_0.ff_layer.ffn_layer2.linear.w (2816, 12, 1024)
max_prefix_decoder.layers.mlp_0.wo.kernel (2816, 12, 1024)

pax_prefix_lm.transformer.repeat.sub.x_layers_0.ff_layer.layer_norm.scale (1024, 12)
max_prefix_decoder.layers.post_self_attention_layer_norm_0.scale (1024, 12)

pax_prefix_lm.transformer.repeat.sub.x_layers_0.layer_norm.scale (1024, 12)
max_prefix_decoder.layers.pre_self_attention_layer_norm_0.scale (1024, 12)

pax_prefix_lm.transformer.repeat.sub.x_layers_0.self_attention.dyn_w_proj.dd (1024, 12, 1, 64)
max_prefix_decoder.layers.self_attention_0.AttentionOp_0.dyn_w_proj.dd.kernel (1024, 12, 1, 64)

pax_prefix_lm.transformer.repeat.sub.x_layers_0.self_attention.dyn_w_proj.dw1 (1024, 12, 1, 4, 64)
max_prefix_decoder.layers.self_attention_0.AttentionOp_0.dyn_w_proj.dw1.kernel (1024, 12, 1, 4, 64)

pax_prefix_lm.transformer.repeat.sub.x_layers_0.self_attention.dyn_w_proj.qkw (1, 12, 4, 64, 4, 16)
max_prefix_decoder.layers.self_attention_0.AttentionOp_0.dyn_w_proj.qkw (1, 12, 4, 64, 4, 16)

pax_prefix_lm.transformer.repeat.sub.x_layers_0.self_attention.k_norm.scale (128, 12)
max_prefix_decoder.layers.self_attention_0.k_norm.scale (128, 12)

pax_prefix_lm.transformer.repeat.sub.x_layers_0.self_attention.key.w (1024, 12, 16, 128)
max_prefix_decoder.layers.self_attention_0.key.kernel (1024, 12, 16, 128)

pax_prefix_lm.transformer.repeat.sub.x_layers_0.self_attention.post.w (16, 12, 128, 1024)
max_prefix_decoder.layers.self_attention_0.out.kernel (16, 12, 128, 1024)

pax_prefix_lm.transformer.repeat.sub.x_layers_0.self_attention.q_norm.scale (128, 12)
max_prefix_decoder.layers.self_attention_0.q_norm.scale (128, 12)

pax_prefix_lm.transformer.repeat.sub.x_layers_0.self_attention.query.w (1024, 12, 16, 128)
max_prefix_decoder.layers.self_attention_0.query.kernel (1024, 12, 16, 128)

pax_prefix_lm.transformer.repeat.sub.x_layers_0.self_attention.value.w (1024, 12, 16, 128)
max_prefix_decoder.layers.self_attention_0.value.kernel (1024, 12, 16, 128)
'''
pax2max = pax2max.split('\n')
pax2max = [l.strip() for l in pax2max if l.strip()]

## params
def replace_prefix(key:str, replace_prefixs:dict, instrings:dict={}, outstrings:dict={}):
    new_key = None
    for prefix, replace_prefix in replace_prefixs.items():
        if instrings and instrings[prefix] not in key:
            continue
        if outstrings and outstrings[prefix] in key:
            continue
        if prefix in key:
            new_key = re.subn(f'{prefix}_', replace_prefix, key, count=1)[0]
            break
    return new_key
    
prefixs = {'pax_prefix': 'state.mdl_vars.params.', 'max_prefix': 'state.params.params.'}
params_pax2max = [replace_prefix(key, prefixs) for key in pax2max]

opt_mv_pax2maxs = []
for mv in ['m', 'v']:
    max_mv = 'mu' if mv == 'm' else 'nu'
    prefixs = {'pax_prefix': f'state.opt_states_0.no_prefix_2.{mv}.params.', 'max_prefix': f'state.opt_state.{max_mv}.params.'}
    outstrings = {'pax_prefix': 'transformer', 'max_prefix': 'layers'}
    opt_mv_pax2max_0 = [replace_prefix(key, prefixs, outstrings=outstrings) for key in pax2max 
                     if replace_prefix(key, prefixs, outstrings=outstrings)]
    
    prefixs = {'pax_prefix': f'state.opt_states_0.p#12#i-1_2.{mv}.params.', 'max_prefix': f'state.opt_state.{max_mv}.params.'}
    instrings = {'pax_prefix': 'transformer', 'max_prefix': 'layers'}
    opt_mv_pax2max_1 = [replace_prefix(key, prefixs, instrings=instrings) for key in pax2max 
                     if replace_prefix(key, prefixs, instrings=instrings)]

    opt_mv_pax2max = opt_mv_pax2max_0 + opt_mv_pax2max_1
    opt_mv_pax2maxs.append(opt_mv_pax2max)

state_pax2max = {'params': params_pax2max, 'm': opt_mv_pax2maxs[0], 'v': opt_mv_pax2maxs[1]}

In [6]:
# 对模型参数进行trannspose
pax_combine_w = {}
for k, v in flat_pax_state.items():
    if 'count' in k or 'step' in k:
        print(f'filter key: {k} =========')
        continue
    ws = v
    transpose = v.shape[0] == layer_num // sub_layer_num
    if transpose:
        ws = ws.transpose(1, 0, *range(2, v.ndim))
    if 'post.w' in k:
        ws = ws.transpose(2, 1, 3, 0)
    print(transpose, k, ws.shape)
    pax_combine_w[k] = ws

False state.mdl_vars.params.lm.embedding_lookup.emb_var (152064, 4096)
False state.mdl_vars.params.lm.final_ln.scale (4096,)
False state.mdl_vars.params.lm.softmax.logits_ffn.linear.w (4096, 152064)
True state.mdl_vars.params.lm.transformer.repeat.sub.x_layers_0.ff_layer.ffn_layer1.linear.w (4096, 12, 5632)
True state.mdl_vars.params.lm.transformer.repeat.sub.x_layers_0.ff_layer.ffn_layer1_gate.linear.w (4096, 12, 5632)
True state.mdl_vars.params.lm.transformer.repeat.sub.x_layers_0.ff_layer.ffn_layer2.linear.w (5632, 12, 4096)
True state.mdl_vars.params.lm.transformer.repeat.sub.x_layers_0.ff_layer.layer_norm.scale (4096, 12)
True state.mdl_vars.params.lm.transformer.repeat.sub.x_layers_0.ff_layer.mgate_layer.linear.w (4096, 12, 44)
True state.mdl_vars.params.lm.transformer.repeat.sub.x_layers_0.layer_norm.scale (4096, 12)
True state.mdl_vars.params.lm.transformer.repeat.sub.x_layers_0.self_attention.dyn_w_proj.dd (4096, 12, 1, 128)
True state.mdl_vars.params.lm.transformer.repeat.sub

In [7]:
# 制作其他层的mapkey
pax2max_keys = {}
for k, pax2max in  state_pax2max.items():
    for i in range(0, len(pax2max) - 1, 2):
        pkey, pshape = pax2max[i].split('(')
        pkey = pkey.strip()
        pshape = '(' + pshape
        mkey, mshape = pax2max[i + 1].split('(')
        mshape = '(' + mshape
        mkey = mkey.strip()
        assert eval(pshape) == eval(mshape)
        pax2max_keys[pkey] = mkey
        if '.repeat.sub.x_layers_0' in pkey:
            for j in range(1, sub_layer_num):
                newpkey = re.subn('x_layers_0', f'x_layers_{j}', pkey, count=1)[0]
                newmkey = re.subn('_0', f'_{j}', mkey, count=1)[0]
                pax2max_keys[newpkey] = newmkey

In [8]:
# 制作其他层的mapkey
pax2max_keys = {}
for k, pax2max in  state_pax2max.items():
    for i in range(0, len(pax2max) - 1, 2):
        pkey, pshape = pax2max[i].split('(')
        pkey = pkey.strip()
        pshape = '(' + pshape
        mkey, mshape = pax2max[i + 1].split('(')
        mshape = '(' + mshape
        mkey = mkey.strip()
        assert eval(pshape) == eval(mshape)
        pax2max_keys[pkey] = mkey
        if '.repeat.sub.x_layers_0' in pkey:
            for i in range(1, sub_layer_num):
                newpkey = re.subn('x_layers_0', f'x_layers_{i}', pkey, count=1)[0]
                newmkey = re.subn('_0', f'_{i}', mkey, count=1)[0]
                pax2max_keys[newpkey] = newmkey

# 基于key map将模型参数进行转换
convert_to_max_w = {}
for k, v in pax_combine_w.items():
    max_key = pax2max_keys[k]
    max_key = tuple(max_key.split('.'))
    convert_to_max_w[max_key] = v
unflat_convert_to_max_w = unflatten_dict(convert_to_max_w)

In [11]:
save_dir = "gs://llm_base_models/maxtext_align_pax_dc/maxtext_align2/checkpoints"
save_dir = "gs://llm_base_models_us-east5/v5p_256/7B/PileDCSlimLlama7B32Kx4x256x1v5p_0705/checkpoints/"
step_format_fixed_length = None
options = orbax.checkpoint.CheckpointManagerOptions()
item = {
    "state": orbax.checkpoint.Checkpointer(orbax.checkpoint.PyTreeCheckpointHandler())
}
max_mngr = orbax.checkpoint.CheckpointManager(save_dir, item, options)

save_step = load_step # 实验随意，转换的是需要设置为load_step

unflat_convert_to_max_w['state']['step'] = np.array(save_step, dtype=np.uint32)
unflat_convert_to_max_w['state']['opt_state']['count'] = np.array(save_step, dtype=np.uint32)
max_mngr.save(save_step, unflat_convert_to_max_w)

True